## 1. Правило ланцюга

$$ Якщо: y = f(g(x))$$
$$\frac{dy}{dx} = \frac{dy}{dg} \cdot \frac{dg}{dx}$$

$$\frac{\partial L}{\partial W_1} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial h_2} \cdot \frac{\partial h_2}{\partial h_1} \cdot \frac{\partial h_1}{\partial W_1}$$


## 2. Функція витрат

### Mean Squared Error (MSE)
$$L = \frac{1}{n} \sum_{i=1}^{n} (y_{pred} - y_{true})^2$$

### Binary Cross-Entropy
$$L = -\frac{1}{n} \sum_{i=1}^{n} \left[ y_{true} \cdot \log(y_{pred}) + (1-y_{true}) \cdot \log(1-y_{pred}) \right]$$

### Cross-Entropy
$$L = -\frac{1}{n} \sum_{i=1}^{n} \sum_{j=1}^{C} y_{true}[i,j] \cdot \log(y_{pred}[i,j])$$



## Backpropagation: покроковий алгоритм

In [ ]:
#Крок 1
def forward_pass(x, W1, b1, W2, b2):
    # Шар 1
    z1 = x @ W1 + b1
    a1 = relu(z1)  # Зберігаємо z1 та a1!

    # Шар 2
    z2 = a1 @ W2 + b2
    a2 = sigmoid(z2)  # Зберігаємо z2 та a2!

    cache = {'z1': z1, 'a1': a1, 'z2': z2, 'a2': a2}
    return a2, cache

#Крок 2
def compute_loss(y_pred, y_true):
    # Binary Cross-Entropy
    loss = -np.mean(y_true * np.log(y_pred) +
                    (1 - y_true) * np.log(1 - y_pred))  #np легко змінюється на torch
    return loss

#Крок 3
def backward_pass(x, y_true, cache, W2):
    m = x.shape[0]  # batch size

    # Градієнт втрат по виходу
    dz2 = cache['a2'] - y_true  # для BCE + sigmoid

    # Градієнти для шару 2
    dW2 = (1/m) * cache['a1'].T @ dz2
    db2 = (1/m) * np.sum(dz2, axis=0, keepdims=True)

    # Градієнт через шар 2
    da1 = dz2 @ W2.T

    # Градієнт через ReLU
    dz1 = da1 * (cache['z1'] > 0)  # ReLU derivative

    # Градієнти для шару 1
    dW1 = (1/m) * x.T @ dz1
    db1 = (1/m) * np.sum(dz1, axis=0, keepdims=True)

    gradients = {'dW2': dW2, 'db2': db2, 'dW1': dW1, 'db1': db1}
    return gradients

In [1]:
# Графічне представлення потоку градієнтів
def visualize_backprop():
    """
    Показує як градієнт "тече" назад через мережу
    """
    # Архітектура: 2 → 3 → 1

    print("Forward Pass:")
    print("x → [W1] → h → [W2] → y → Loss")
    print("         ↓      ↓       ↓")
    print("    зберігаємо всі значення")

    print("\nBackward Pass:")
    print("∂L/∂W1 ← ∂L/∂h ← ∂L/∂y ← ∂L/∂L=1")
    print("   ↑        ↑       ↑")
    print("chain    chain   chain")
    print("rule     rule    rule")

visualize_backprop()

Forward Pass:
x → [W1] → h → [W2] → y → Loss
         ↓      ↓       ↓
    зберігаємо всі значення

Backward Pass:
∂L/∂W1 ← ∂L/∂h ← ∂L/∂y ← ∂L/∂L=1
   ↑        ↑       ↑
chain    chain   chain
rule     rule    rule


# 3. Оптимізатори

### Vanilla SGD
$$\theta_{t+1} = \theta_t - \eta \cdot \nabla_{\theta} L(\theta_t)$$

### SGD with Momentum
$$v_{t+1} = \beta \cdot v_t - \eta \cdot \nabla_{\theta} L(\theta_t)$$

$$\theta_{t+1} = \theta_t + v_{t+1}$$

### RMSprop
$$s_{t+1} = \beta \cdot s_t + (1 - \beta) \cdot (\nabla_{\theta} L)^2$$

$$\theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{s_{t+1} + \epsilon}} \cdot \nabla_{\theta} L$$

### Adam
$$m_{t+1} = \beta_1 \cdot m_t + (1 - \beta_1) \cdot \nabla_{\theta} L$$

$$v_{t+1} = \beta_2 \cdot v_t + (1 - \beta_2) \cdot (\nabla_{\theta} L)^2$$

$$\hat{m}_{t+1} = \frac{m_{t+1}}{1 - \beta_1^{t+1}}$$

$$\hat{v}_{t+1} = \frac{v_{t+1}}{1 - \beta_2^{t+1}}$$

$$\theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{\hat{v}_{t+1}} + \epsilon} \cdot \hat{m}_{t+1}$$



In [2]:
class SGD:
    def __init__(self, learning_rate=0.01):
        self.lr = learning_rate

    def update(self, params, grads):
        for param, grad in zip(params, grads):
            param -= self.lr * grad

class SGDMomentum:
    def __init__(self, learning_rate=0.01, momentum=0.9):
        self.lr = learning_rate
        self.momentum = momentum
        self.velocity = None

    def update(self, params, grads):
        if self.velocity is None:
            self.velocity = [np.zeros_like(p) for p in params]

        for i, (param, grad) in enumerate(zip(params, grads)):
            self.velocity[i] = self.momentum * self.velocity[i] - self.lr * grad
            param += self.velocity[i]

class RMSprop:
    def __init__(self, learning_rate=0.001, beta=0.9, epsilon=1e-8):
        self.lr = learning_rate
        self.beta = beta
        self.epsilon = epsilon
        self.cache = None

    def update(self, params, grads):
        if self.cache is None:
            self.cache = [np.zeros_like(p) for p in params]

        for i, (param, grad) in enumerate(zip(params, grads)):
            self.cache[i] = self.beta * self.cache[i] + (1 - self.beta) * grad**2
            param -= self.lr * grad / (np.sqrt(self.cache[i]) + self.epsilon)

class Adam:
    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.lr = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = None
        self.v = None
        self.t = 0

    def update(self, params, grads):
        if self.m is None:
            self.m = [np.zeros_like(p) for p in params]
            self.v = [np.zeros_like(p) for p in params]

        self.t += 1

        for i, (param, grad) in enumerate(zip(params, grads)):
            # Momentum
            self.m[i] = self.beta1 * self.m[i] + (1 - self.beta1) * grad
            # RMSprop
            self.v[i] = self.beta2 * self.v[i] + (1 - self.beta2) * grad**2

            # Bias correction
            m_hat = self.m[i] / (1 - self.beta1**self.t)
            v_hat = self.v[i] / (1 - self.beta2**self.t)

            # Update
            param -= self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)

# 4: Learning Rate Scheduling

### Step Decay
$$\eta_t = \eta_0 \cdot \gamma^{\lfloor \frac{t}{s} \rfloor}$$
де
s - розмір кроку,
γ - коефіцієнт зменшення

### Exponential Decay
$$\eta_t = \eta_0 \cdot \gamma^t$$

### Cosine Annealing
$$\eta_t = \eta_{min} + \frac{1}{2}(\eta_{max} - \eta_{min})\left(1 + \cos\left(\frac{\pi \cdot t}{T_{max}}\right)\right)$$

### Linear Warmup
$$\eta_t = \begin{cases}
\frac{t}{T_{warmup}} \cdot \eta_{target}, & \text{if } t < T_{warmup} \\
\eta_{target}, & \text{otherwise}
\end{cases}$$


In [ ]:
def find_learning_rate(model, data_loader, start_lr=1e-7, end_lr=10):
    """
    Техніка для знаходження оптимального learning rate
    """
    lrs = []
    losses = []

    lr = start_lr

    for batch in data_loader:
        # Forward pass
        loss = model.compute_loss(batch)
        losses.append(loss)
        lrs.append(lr)

        # Backward pass з поточним lr
        model.backward()
        model.update_weights(lr)

        # Збільшуємо lr
        lr *= 1.1

        if lr > end_lr or loss > losses[0] * 4:
            break

    # Оптимальний lr - де loss падає найшвидше
    # (найбільший нахил на графіку)
    return lrs, losses

# Градієнти

### Градієнт MSE
$$\frac{\partial L_{MSE}}{\partial y_{pred}} = \frac{2}{n}(y_{pred} - y_{true})$$

### Градієнт BCE
$$\frac{\partial L_{BCE}}{\partial y_{pred}} = -\frac{1}{n}\left(\frac{y_{true}}{y_{pred}} - \frac{1-y_{true}}{1-y_{pred}}\right)$$

### Градієнт Softmax + Cross-Entropy (спрощений)
$$\frac{\partial L}{\partial z_i} = y_{pred,i} - y_{true,i}$$

### Gradient Clipping
$$g_{clipped} = \begin{cases}
g, & \text{if } ||g|| \leq \text{max\_norm} \\
\frac{\text{max\_norm}}{||g||} \cdot g, & \text{otherwise}
\end{cases}$$

